In [61]:
%load_ext autoreload
%autoreload 2

import sys, os
from pathlib import Path
import plotly.graph_objects as go
from typing import Iterable

# Fix import
sys.path.append(os.path.abspath(os.path.join('..')))
from facilitators import BarFacilitator, StackedQuantityEvolutionFacilitatorBase
from data import read_sol_keys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
PROJECT_ROOT_DIR = Path().resolve().parent
SOLUTION_DIR = PROJECT_ROOT_DIR / "solution_files3"

DAILY_DATA = SOLUTION_DIR / "solution_daily_d10_h6_s6.sol"
SEASON_DATA = SOLUTION_DIR / "solution_seasonal_free_d40_h6_s6_6.sol"

WRITE_DIR = PROJECT_ROOT_DIR / "generated_figs2"
WRITE_FIGS = True

REGION = "ALL"
YEAR = 2050

In [22]:
print("Keys...")

for key in read_sol_keys(SEASON_DATA):
    if "Storage" in key:
        print(key)

Keys...
SalvageValueStorage
TotalDiscountedStorageCost
StorageLevelYearFinish
StorageLevelTSStart
DiscountedSalvageValueStorage
StorageLevelYearStart
NewStorageCapacity
DiscountedCapitalInvestmentStorage
CapitalInvestmentStorage
AccumulatedNewStorageCapacity


In [63]:

def stacked_capital_invest(sol_path: str, region: str = "ALL", title_addon: str = None, write_fig = False) -> go.Figure:
    storage_evolution = StackedQuantityEvolutionFacilitatorBase(sol_path=sol_path, type_of_data_to_read="CapitalInvestment", 
                                                                region=region, extra_identifying_columns=["Technology"])

    d = storage_evolution.get_relevant_data()
    d.filter_by_containing_string(column="Technology", identifier="D_")

    if region == "ALL":
        d.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    # Remove those who are 0 for all years
    techs_over_zero = d.df[d.df["Value"] > 0]["Technology"].unique()
    d.filter_by_list(column="Technology", identifier_list=techs_over_zero)

    fig = storage_evolution.plot(storage_evolution.generate_traces(d))

    fig.update_layout(
        width=1400,
        height=800,
        title=f"Capital Investments (Euro), Region: {region}" + f", {title_addon}" if title_addon is not None else "",
        yaxis_title="M€",
        xaxis_title="Year",
    )

    if write_fig:
        fig.write_image(WRITE_DIR / f"stacked_capital_invest_{region}_{title_addon}.png")

    return fig

In [60]:
stacked_capital_invest(DAILY_DATA, region=REGION, title_addon="Daily", write_fig=WRITE_FIGS).show()
stacked_capital_invest(SEASON_DATA, region=REGION, title_addon="Season", write_fig=WRITE_FIGS).show()

No unit conversion applied!


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\mortensv\\OneDrive - SINTEF\\Dokumenter\\forked_scenario_dashboard\\solution_files2\\solution_seasonal_free_d40_h6_s6_6.sol'

In [64]:
def stacked_accumulated_energy(sol_path: str, region: str = "ALL", title_addon: str = None, write_fig = False) -> tuple[go.Figure, Iterable[str]]:
    storage_evolution = StackedQuantityEvolutionFacilitatorBase(sol_path=sol_path, type_of_data_to_read="NewStorageCapacity", 
                                                                region=region, extra_identifying_columns=["Storage"])

    d = storage_evolution.get_relevant_data()
    #d.filter_by_containing_string(column="Technology", identifier="D_")

    if region == "ALL":
        d.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    # Remove those who are 0 for all years
    techs_over_zero = d.df[d.df["Value"] > 0]["Storage"].unique()
    d.filter_by_list(column="Storage", identifier_list=techs_over_zero)

    # Do cumsum over years
    d.df["Value"] = d.df.groupby("Storage")["Value"].cumsum()

    fig = storage_evolution.plot(storage_evolution.generate_traces(d))

    fig.update_layout(
        width=1400,
        height=800,
        title=f"Accumulated Storage Capacity, Region: {region}" + f", {title_addon}" if title_addon is not None else "",
        yaxis_title="TWh",
        xaxis_title="Year",
    )

    if write_fig:
        fig.write_image(WRITE_DIR / f"stacked_accumulated_storage_energy_{region}_{title_addon}.png")

    return fig, techs_over_zero

In [66]:
#stacked_accumulated_energy(DAILY_DATA, region=REGION, title_addon="Daily", write_fig=WRITE_FIGS)[0].show()
fig, s_over_zero = stacked_accumulated_energy(SEASON_DATA, region=REGION, title_addon="Season", write_fig=WRITE_FIGS)
fig.show()

Converting from PetaJoules to TerraWattHours


In [27]:
def bar_capacity_evolution_fig(tech: str, region: str, daily_data: Path, season_data: Path, write_fig: bool = False) -> go.Figure:

    bar = BarFacilitator(sol_paths={"daily": daily_data, "season": season_data}, type_of_data_to_read="NewStorageCapacity", 
                        x_grouping_columns=["Year", "Source"], legend_grouping_columns=["Storage", "Source"])

    data = bar.get_relevant_data()

    #data.filter_by_list(column="Technology", identifier_list=["D_Battery_Li-Ion"])
    data.filter_by_identifier(column="Storage", identifier=tech)

    if region == "ALL":
        data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    data.filter_by_identifier(column="Region", identifier=region)

    fig = bar.plot(bar.generate_traces(data))

    fig.update_layout(
        title="New Storage Capacities (Energy)",
        xaxis_title="Year",
        yaxis_title="TWh",
        width=1200,
        height=600
        )
    
    if write_fig:
        fig.write_image(WRITE_DIR / f"bar_energy_capacity_evolution_{tech}_{region}.png")

    return fig

In [28]:
for s in s_over_zero:
    bar_capacity_evolution_fig(s, region=REGION, daily_data=DAILY_DATA, season_data=SEASON_DATA, write_fig=WRITE_FIGS).show()

Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours


# Code

In [29]:
# StorageLevelTSStart
def bar_storage_level_evolution_fig(tech: str, region: str, daily_data: Path, season_data: Path, write_fig: bool = False) -> go.Figure:

    bar = BarFacilitator(sol_paths={"daily": daily_data, "season": season_data}, type_of_data_to_read="NewStorageCapacity", 
                        x_grouping_columns=["Year", "Source"], legend_grouping_columns=["Storage", "Source"])

    data = bar.get_relevant_data()

    #data.filter_by_list(column="Technology", identifier_list=["D_Battery_Li-Ion"])
    data.filter_by_identifier(column="Storage", identifier=tech)

    if region == "ALL":
        data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    data.filter_by_identifier(column="Region", identifier=region)

    fig = bar.plot(bar.generate_traces(data))

    fig.update_layout(
        title="New Storage Capacities (Energy)",
        xaxis_title="Year",
        yaxis_title="TWh",
        width=1200,
        height=600
        )
    
    if write_fig:
        fig.write_image(WRITE_DIR / f"bar_energy_capacity_evolution_{tech}_{region}.png")

    return fig

In [30]:
def hourly_rate_of_activity_fig(tech: str, region: str, year: int, daily_data: Path, season_data: Path, write_fig: bool = False) -> go.Figure:

    bar = BarFacilitator(sol_paths={"daily": daily_data, "season": season_data}, type_of_data_to_read="StorageLevelTSStart", 
                        x_grouping_columns=["TS", "Source"], legend_grouping_columns=["Storage", "Source"])

    data = bar.get_relevant_data()

    # Fix yearsplit division
    #data.df["Value"] *= data.year_split

    # Keep only tech
    data.filter_by_identifier(column="Storage", identifier=tech)

    # Keep only modes 1 and 2
    #data.filter_by_list("Mode", [1, 2])

    # Set mode 1 to negative
    #data.df.loc[data.df["Mode"] == 2, "Value"] *= -1

    # Aggregate over Mode
    #data.aggreagate_all_by_sum(column_to_aggregate="Mode", aggregated_entry_name=-1, column_to_sum="Value")

    # Get unique timesteps, and get the one hitting daily stuff
    timesteps = data.df["TS"].unique()
    timesteps.sort()
    hourstep = timesteps[1] - timesteps[0]
    start_hour = timesteps[0]

    zero_timesteps = []
    for t in timesteps:
        if (t - hourstep + start_hour) % 24 == 0:
            zero_timesteps.append(t)

    print("Zero timesteps", zero_timesteps)

    if region == "ALL":
        data.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    data.df = data.df[data.df["Year"] == year]

    traces = bar.generate_traces(data)

    traces.append(go.Scatter(x=[zero_timesteps, ["daily"]*len(zero_timesteps)], y=[0]*len(zero_timesteps), 
                             mode="markers", showlegend = True, name="Daily reset",
                             marker=dict(color="red", size=10)))

    fig = bar.plot(traces)

    fig.update_layout(
            title=f"Storage Levels, Region: {region}, Year: {year}",
            yaxis_title="TWh",
            xaxis_title="Year",
            width=1200,
            height=600
        )
    
    if write_fig:
        print("Writing fig")
        fig.write_image(WRITE_DIR / f"hourly_storage_levels_{tech}_{region}_{year}.png")

    return fig

In [31]:
hourly_rate_of_activity_fig("S_Battery_Li-Ion", region=REGION, year=YEAR, daily_data=DAILY_DATA, season_data=SEASON_DATA, write_fig=WRITE_FIGS).show()
hourly_rate_of_activity_fig("S_Heat_HLI", region=REGION, year=YEAR, daily_data=DAILY_DATA, season_data=SEASON_DATA, write_fig=WRITE_FIGS).show()

Converting from PetaJoules to TerraWattHours
Zero timesteps [np.int64(744), np.int64(1728), np.int64(2712), np.int64(3696), np.int64(4680), np.int64(5664), np.int64(6648), np.int64(7632), np.int64(8616)]
Writing fig


Converting from PetaJoules to TerraWattHours
Zero timesteps [np.int64(744), np.int64(1728), np.int64(2712), np.int64(3696), np.int64(4680), np.int64(5664), np.int64(6648), np.int64(7632), np.int64(8616)]
Writing fig
